###### 使用一个大模型生成两份简历

In [1]:
_zhaoyuxuan = """
姓名：赵宇轩

性别：男

出生日期：1995年8月8日

一、童年时期（1995-2005）

1995年，赵宇轩出生于我国风景秀丽的江南水乡——云水市，父亲赵志强是云水市一家大型民营企业——蓝海集团的总经理，母亲林雅是一名中学教师。
2005年，赵宇轩在云水市实验小学毕业，成绩优异。在校期间，他热爱足球、钢琴和围棋，多次代表学校参加比赛并获奖。
二、中学时期（2005-2013）

2005年，赵宇轩进入云水市第一中学，开始了初中生活。他加入了学校足球队，担任队长，带领球队获得全市中学生足球联赛冠军。
2013年，赵宇轩以全市中考状元的身份考入著名的翠微中学。高中期间，他担任学生会副主席，组织策划了多项校园活动。
三、大学时期（2013-2017）

2013年，赵宇轩进入我国顶尖学府——华夏大学，攻读计算机科学与技术专业。在校期间，他加入了校足球队，并担任队长。
2017年，赵宇轩以优异的成绩毕业，获得计算机科学与技术学士学位。同年，他获得全额奖学金，赴海外名校——牛津理工学院攻读人工智能硕士学位。
四、留学时期（2017-2019）

2017年，赵宇轩抵达英国，开始了留学生活。在牛津理工学院，他发表了多篇关于人工智能领域的学术论文。
2019年，赵宇轩获得人工智能硕士学位，并被评为优秀毕业生。同年，他回国加入了一家创新型科技公司——智慧星辰科技有限公司。
五、职业生涯（2019-至今）

2019年，赵宇轩加入智慧星辰科技有限公司，担任人工智能研发工程师。他带领团队成功研发出多款智能产品，为公司创造了巨大效益。
2021年，赵宇轩晋升为智慧星辰科技有限公司研发部门经理，负责公司人工智能项目的研发管理工作。
2023年，赵宇轩被评为“全国杰出青年科技工作者”，并在同年获得“星河科技奖”。
至今，赵宇轩继续在人工智能领域深耕，为我国科技创新贡献力量。同时，他关注环保事业，积极参与公益活动，助力可持续发展。
"""
_sunyuexing = """
姓名：孙悦心

性别：女

出生日期：1992年3月12日

一、童年时期（1992-2002）

1992年，孙悦心出生于我国西部的一个历史名城——锦城，父亲孙宏伟是一名建筑师，母亲刘晓梅是一名图书管理员。
2002年，孙悦心在锦城实验小学毕业，她自幼对文学和艺术有着浓厚的兴趣，曾多次在学校的作文比赛和绘画比赛中获奖。
二、中学时期（2002-2010）

2002年，孙悦心进入锦城实验中学，开始了她的初中生活。她加入了学校的文学社，并担任社长，组织了一系列文学活动。
2010年，孙悦心以优异的成绩考入锦城第一高级中学，高中期间，她专注于文科学习，同时参加了学校的戏剧社，并在多次戏剧比赛中担任主角。
三、大学时期（2010-2014）

2010年，孙悦心考入我国著名的文艺学府——彩虹艺术学院，攻读戏剧影视文学专业。在校期间，她积极参与校园戏剧创作和表演。
2014年，孙悦心以优秀的毕业作品获得戏剧影视文学学士学位，并在毕业大戏中担任主演，获得了师生的一致好评。
四、职业生涯（2014-至今）

2014年，孙悦心加入了一家知名影视制作公司——星光传媒，开始了她的职业生涯，担任剧本编辑和演员。
2016年，孙悦心参与编剧的电视剧《岁月无声》获得了年度最佳剧本奖，她本人也因此获得业界的关注。
2018年，孙悦心转型成为导演，执导了自己的第一部电影《梦回长安》，该电影在国内外电影节上获得了多个奖项。
2021年，孙悦心成立了属于自己的影视工作室——悦心影业，致力于创作更多优秀的影视作品。
至今，孙悦心在影视行业中的影响力日益扩大，她的作品深受观众喜爱。同时，她也是一位公益活动的积极参与者，经常为弱势群体发声，传递正能量。
"""

##### 创建文档

In [2]:
from langchain_core.documents import Document

_docs = [
    Document(page_content=_zhaoyuxuan),
    Document(page_content=_sunyuexing)
]

##### 导入模型

In [3]:
from langchain_openai import ChatOpenAI
_llm=ChatOpenAI(
    api_key="ollama", #ollama api_key随便写
    model="qwen2.5:7b", #看ollama中的名
    base_url="http://192.168.10.11:60006/v1",#注意后面加v1
    temperature=0.7
)

##### 加载一个embeeding模型，导入AI-ModelScope/bge-large-zh-v1.5 ，从modelscope复制

In [4]:
from modelscope import snapshot_download
model_dir = snapshot_download("AI-ModelScope/bge-large-zh-v1.5", revision='master')

from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = model_dir
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
_embed = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
_embed.query_instruction = "为这个句子生成表示以用于检索相关文章："

/root/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-11 02:22:21,224 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


In [15]:
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter #文档分割器
# 将文档进行拆分
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20) #按100进行拆分，重叠20
splits = text_splitter.split_documents(_docs)
# 制作向量数据库
_vectorstore = Chroma.from_documents(splits,_embed)
# 进行查询，使用相似度进行搜索
_vectorstore.similarity_search_with_score("谁是理工男",k=1) #使用距离进行匹配，只要1个

[(Document(metadata={}, page_content='2017年，赵宇轩以优异的成绩毕业，获得计算机科学与技术学士学位。同年，他获得全额奖学金，赴海外名校——牛津理工学院攻读人工智能硕士学位。\n四、留学时期（2017-2019）'),
  1.2074875831604004)]

##### 使用模板进行提问

In [19]:
from langchain.prompts import ChatPromptTemplate
_prompt ="""
参考以下的内容：{ref}
回答以下问题：{question}
"""
_messages = ChatPromptTemplate([
    ("system","使用中文帮助用户"),
    ("human",_prompt)
])

In [16]:
# 将函数转成runnables形式
# _retriver=_vectorstore.as_retriever().bind(k=2)

RunnableBinding(bound=VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7ff081203eb0>, search_kwargs={}), kwargs={'k': 2}, config={}, config_factories=[])

##### 将函数转成runnables形式

In [17]:
from langchain_core.runnables import RunnableLambda
_retriver = RunnableLambda(_vectorstore.similarity_search_with_score).bind(k=2)
print(_retriver)

bound=RunnableLambda(similarity_search_with_score) kwargs={'k': 2} config={} config_factories=[]


##### 创建链

In [25]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

_chain = {"question":RunnablePassthrough(),"ref":_retriver}|_messages|_llm|StrOutputParser()

In [24]:
_chain.invoke("孙悦心是哪个小学毕业的？")

AIMessage(content='孙悦心是在锦城实验小学毕业的。根据提供的信息，她在2002年毕业于锦城实验小学，并且自幼对文学和艺术有着浓厚的兴趣，曾多次在校内比赛中获奖。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 198, 'total_tokens': 244, 'completion_tokens_details': None}, 'model_name': 'qwen2.5:7b', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-37f9d010-d5f1-4b71-923b-4f54a16a372e-0', usage_metadata={'input_tokens': 198, 'output_tokens': 46, 'total_tokens': 244, 'input_token_details': {}, 'output_token_details': {}})